In [1]:
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

In [2]:
# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [3]:
# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

# Viz
#import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#combined_df = pd.read_csv(f'data/reviews_hc2/hackathon2/test_train_combined.csv')
train_df = pd.read_csv('/home/FRACTAL/jitesh.arora/course-v3/nbs/dl1/data/reviews_hc2/hackathon2/train_df_final.csv')
test_df = pd.read_csv('/home/FRACTAL/jitesh.arora/course-v3/nbs/dl1/data/reviews_hc2/hackathon2/translated_test_correct.csv')

print(train_df.shape)
print(test_df.shape)
                      

(129726, 20)
(3645, 47)


/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
reduced_df = combined_df.loc[:,combined_df.apply(pd.Series.nunique) != 1]
print(reduced_df.shape)

(136983, 21)


In [6]:
reduced_df = reduced_df.loc[:,reduced_df.apply(pd.Series.nunique) != 0]
print(reduced_df.shape)

(136983, 19)


In [7]:
reduced_df.nunique()

author                   84670
crawled                 129724
published                70302
text                    128728
thread.country               7
thread.published         70302
thread.section_title     12933
thread.site_full            25
thread.site_section      11454
thread.spam_score         1095
thread.title             84668
thread.title_full       117671
thread.url              129726
thread.uuid             129726
title                    84668
url                     129726
uuid                    129726
target                       2
is_test                      2
dtype: int64

In [8]:
df = reduced_df.copy()

In [9]:
test_df = df[df['is_test']==1]
train_df = df[df['is_test']==0]
print(test_df.shape)
print(train_df.shape)

(7257, 19)
(129726, 19)


In [13]:
test_df['thread.country'].value_counts(normalize  = True)

IE    0.445776
US    0.444812
GB    0.050021
FR    0.031005
SE    0.013366
NL    0.012677
DK    0.002343
Name: thread.country, dtype: float64

In [14]:
train_df['thread.country'].value_counts(normalize  = True)

US    0.436505
IE    0.395094
GB    0.085272
FR    0.045766
NL    0.018516
SE    0.015856
DK    0.002991
Name: thread.country, dtype: float64

In [15]:
train_df['is_valid'] = 0
idx1 = np.array(train_df[train_df['target']==1].sample(frac= 0.1 , random_state = 42)['is_valid'].index.values).tolist()# tol
idx2 = (train_df[train_df['target']==0].sample(frac= 0.1 , random_state = 42)['is_valid'].index.values).tolist()

val_idxs = idx1+ idx2
print(len(val_idxs))

train_df['is_valid'] = train_df.index.isin(val_idxs)
train_df['is_valid'] =  train_df['is_valid'].apply(lambda x: 1 if x== True else 0)

/home/FRACTAL/jitesh.arora/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


12973


/home/FRACTAL/jitesh.arora/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/FRACTAL/jitesh.arora/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
train_df.reset_index(drop=True , inplace=True)
test_df.reset_index(drop=True , inplace=True)

In [17]:
train_df.drop(columns=['published' , 'title' , 'uuid' , 'url'] , inplace=True)
test_df.drop(columns=['published' , 'title' , 'uuid' , 'url'] , inplace=True)

/home/FRACTAL/jitesh.arora/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
train_df.isnull().sum()

author                  11
crawled                  0
text                     0
thread.country           0
thread.published         0
thread.section_title     0
thread.site_full         0
thread.site_section      0
thread.spam_score        0
thread.title            26
thread.title_full        0
thread.url               0
thread.uuid              0
target                   0
is_test                  0
is_valid                 0
dtype: int64

In [19]:
train_df = train_df.fillna('null')
test_df = test_df.fillna('null')

In [20]:
def new_fn(x):
    try :
        x = str(x).split('?')[0]
    except:
        x = str(x)
    return x

In [21]:
train_df['thread.site_section']= train_df['thread.site_section'].apply(lambda x : str(x).split('/')[-1])
train_df['thread.site_section'] = train_df['thread.site_section'].apply(lambda x : new_fn(x))

test_df['thread.site_section']= test_df['thread.site_section'].apply(lambda x : str(x).split('/')[-1])
test_df['thread.site_section'] = test_df['thread.site_section'].apply(lambda x : new_fn(x))

In [22]:
train_df.columns

Index(['author', 'crawled', 'text', 'thread.country', 'thread.published',
       'thread.section_title', 'thread.site_full', 'thread.site_section',
       'thread.spam_score', 'thread.title', 'thread.title_full', 'thread.url',
       'thread.uuid', 'target', 'is_test', 'is_valid'],
      dtype='object')

In [23]:
def rename(x):
    try:
        x = str(x).split('.')[-1]
    except:
        x = str(x)
    return x

In [24]:
train_df.rename(columns= lambda x : rename(x) , inplace=True)
test_df.rename(columns= lambda x : rename(x) , inplace=True)


In [25]:

train_df['crawled'] =  train_df['crawled'].apply(lambda x: str(x)[0:10])
train_df['published'] =  train_df['published'].apply(lambda x: str(x)[0:10])

test_df['crawled'] =  test_df['crawled'].apply(lambda x: str(x)[0:10])
test_df['published'] =  test_df['published'].apply(lambda x: str(x)[0:10])

In [26]:
train_df.nunique()

author            84671
crawled              54
text             128727
country               7
published            86
section_title     12933
site_full            25
site_section       4033
spam_score         1089
title             84669
title_full       117670
url              129725
uuid             129725
target                2
is_test               1
is_valid              2
dtype: int64

In [27]:
print(train_df.shape)
print(test_df.shape)

(129726, 16)
(7257, 15)


In [30]:
train_df.to_csv('/home/FRACTAL/jitesh.arora/course-v3/nbs/dl1/data/reviews_hc2/hackathon2/train_processed.csv' , index = False)
test_df.to_csv('/home/FRACTAL/jitesh.arora/course-v3/nbs/dl1/data/reviews_hc2/hackathon2/test_processed.csv' , index = False)